# X-ray photoelectron spectroscopy (XPS)
For the source code, see [xps](https://workgraph-collections.readthedocs.io/en/latest/qe/module.html#workgraph_collections.ase.espresso.xps.xps_workgraph).


## Visualizing the WorkGraph Builder


In [1]:
from workgraph_collections.qe.xps import xps_workgraph
from aiida import load_profile
load_profile()

node = xps_workgraph.node()
node.to_html()

## Visualizing the WorkGraph

In [2]:
from workgraph_collections.qe.xps import xps_workgraph
from aiida import load_profile
load_profile()

wg = xps_workgraph()
wg.to_html()

## Example: ETFA molecule

### Prepare the inputs and submit the workflow


In [3]:
from ase.io import read
from aiida import load_profile
from aiida.orm import (
    Dict,
    KpointsData,
    StructureData,
    load_code,
    load_group,
    QueryBuilder,
    Group,
)
from workgraph_collections.qe.xps import xps_workgraph

load_profile()

def load_core_hole_pseudos(pseudo_group="pseudo_demo_pbe"):
    """Load the core hole pseudos."""
    pseudo_group = (
        QueryBuilder().append(Group, filters={"label": pseudo_group}).one()[0]
    )
    pseudos = {"core_hole": {
        pseudo.label.split("_")[0]: pseudo for pseudo in pseudo_group.nodes if "gs" not in pseudo.label
    },
    "gipaw": {
        pseudo.label.split("_")[0]: pseudo for pseudo in pseudo_group.nodes if "gs" in pseudo.label
    }}
    return pseudos, pseudo_group.base.extras.get("correction", {})


# create input structure node
etfa = read("datas/ETFA.xyz")
structure = StructureData(ase=etfa)
# create the PW node
# code = load_code("qe-7.2-pw@localhost")
code = load_code("qe-7.2-pw@eiger")
parameters = Dict(
    {
        "CONTROL": {
            "calculation": "scf",
        },
        "SYSTEM": {
            "ecutwfc": 50,
            "ecutrho": 600,
            "occupations": "fixed",
        },
    }
)
kpoints = KpointsData()
kpoints.set_kpoints_mesh([1, 1, 1])
# Load the pseudopotential family.
core_hole_pseudos, correction_energies = load_core_hole_pseudos("pseudo_demo_pbe")
correction_energies = {
    key.split("_")[0]: value["core"] for key, value in correction_energies.items()
}
pseudo_family = load_group("SSSP/1.3/PBE/efficiency")
pseudos = pseudo_family.get_pseudos(structure=structure)
#
metadata = {
    "options": {
        "resources": {
            "num_machines": 1,
            "num_mpiprocs_per_machine": 1,
        },
    }
}
metadata_eiger = {
    "options": {
        "custom_scheduler_commands": "#SBATCH --account=mr32",
        "resources": {
            "num_machines": 1,
            "num_mpiprocs_per_machine": 36,
        },
    }
}
# ===============================================================================
wg = xps_workgraph(
    structure=structure,
    code=code,
    # atoms_list=[0, 1, 2, 3],
    element_list=["C"],
    parameters=parameters,
    kpoints=kpoints,
    pseudos=pseudos,
    is_molecule=True,
    core_hole_pseudos=core_hole_pseudos,
    correction_energies=correction_energies,
    metadata=metadata_eiger,
)
wg.name = "QE, XPS, ETFA"
print("core_hole_pseudos", core_hole_pseudos.keys())
# wg.submit()
wg.run()


core_hole_pseudos dict_keys(['core_hole', 'gipaw'])


06/09/2024 08:36:17 PM <642816> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [56713|WorkGraphEngine|continue_workgraph]: Continue workgraph.


Continue workgraph.


06/09/2024 08:36:18 PM <642816> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [56713|WorkGraphEngine|continue_workgraph]: nodes ready to run: marked_structures
06/09/2024 08:36:18 PM <642816> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [56713|WorkGraphEngine|run_nodes]: Run node: marked_structures, type: CALCFUNCTION


------------------------------------------------------------
executor:  <function get_xspectra_structures at 0x716b5ca7dbc0>
args:  [<StructureData: uuid: e606cdac-7c3c-417c-8c39-469d54365ec9 (pk: 56707)>]
kwargs:  {'structure': <StructureData: uuid: e606cdac-7c3c-417c-8c39-469d54365ec9 (pk: 56707)>}
var_kwargs:  {'absorbing_elements_list': <List: uuid: e027cebe-04b2-4eab-810b-d1c45034b843 (pk: 56708) value: ['C']>, 'is_molecule_input': <Bool: uuid: c224896a-119d-42dc-94aa-3098d355a629 (pk: 56709) value: True>}
node type: calcfunction/workfunction.
link_label:  supercell uuid: 80ba80df-033f-4df1-a12e-96e773cf469e (unstored)
link_label:  site_0_C uuid: 06d9cb5e-22fd-4446-99ca-55763cb21075 (unstored)
link_label:  site_1_C uuid: 946cda02-6b69-41fb-b782-33e72c103463 (unstored)
link_label:  site_2_C uuid: 93a7f087-1cc6-4aec-baad-2ef34f04c183 (unstored)
link_label:  site_3_C uuid: 2a0dd23d-f52c-4004-885b-b46e5d581c1d (unstored)


06/09/2024 08:36:18 PM <642816> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [56713|WorkGraphEngine|run_nodes]: Node: marked_structures finished.
06/09/2024 08:36:18 PM <642816> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [56713|WorkGraphEngine|continue_workgraph]: Continue workgraph.


link_label:  output_parameters uuid: 8964f17f-8bb3-46ab-807e-0996ba3fddaf (unstored)
Continue workgraph.


06/09/2024 08:36:18 PM <642816> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [56713|WorkGraphEngine|continue_workgraph]: nodes ready to run: run_scf
06/09/2024 08:36:18 PM <642816> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [56713|WorkGraphEngine|run_nodes]: Run node: run_scf, type: graph_builder
06/09/2024 08:36:19 PM <642816> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [56713|WorkGraphEngine|on_except]: Traceback (most recent call last):
  File "/home/xing/miniconda3/envs/aiida/lib/python3.11/site-packages/plumpy/process_states.py", line 228, in execute
    result = self.run_fn(*self.args, **self.kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/xing/repos/superstar54/aiida-workgraph/aiida_workgraph/engine/workgraph.py", line 267, in run
    return self._do_step()
           ^^^^^^^^^^^^^^^
  File "/home/xing/repos/superstar54/aiida-workgraph/aiida_workgraph/engine/workgraph.py", line 282, in

------------------------------------------------------------
executor:  <function run_scf at 0x716b58e665c0>
args:  []
kwargs:  {'structure': <StructureData: uuid: e606cdac-7c3c-417c-8c39-469d54365ec9 (pk: 56707)>, 'code': <InstalledCode: Remote code 'qe-7.2-pw' on eiger pk: 56609, uuid: a6efafe4-b2ec-4f72-9c06-ecc412a6ddfd>, 'parameters': <Dict: uuid: f9997682-e2af-401e-bedf-f74be625b676 (pk: 56710)>, 'kpoints': <KpointsData: uuid: b347bc2c-cb7c-4d75-bb97-767e08b1e91d (pk: 56711)>, 'pseudos': {'C': <UpfData: uuid: acc0db68-1169-4fc6-8f6b-c19a961739a2 (pk: 531)>, 'F': <UpfData: uuid: c656bc1d-e22e-4fd2-b13f-577a9eface5f (pk: 615)>, 'H': <UpfData: uuid: e21ed402-1c7f-492d-ac2b-7a31f61e891d (pk: 592)>, 'O': <UpfData: uuid: 7ab4de29-5f82-4382-8d39-883fb29c61c7 (pk: 612)>}, 'core_hole_pseudos': {'core_hole': {'Al': <UpfData: uuid: 3225e978-5e40-480a-8cc9-23551bb206ed (pk: 6996)>, 'Au': <UpfData: uuid: b360facb-8723-422a-8380-3937099eddb1 (pk: 6989)>, 'C': <UpfData: uuid: 2ed40ad1-57b5-46d2

KeyError: 'site_0_C'